In [ ]:

import os
import numpy as np
import pandas as pd
x = pd.read_excel ('data_train.xlsx')
test=pd.read_excel('data_test.xlsx')
print(test.columns)

In [ ]:
y=x.pop('Price')
x.describe()

In [ ]:
x['Name'] = x.Name.str.split().str.get(0)
test['Name'] = test.Name.str.split().str.get(0)

In [ ]:
x.isna().sum()


In [ ]:
x['Seats'].fillna(x['Seats'].mean(),inplace=True)
test['Seats'].fillna(test['Seats'].mean(),inplace=True)
data = pd.concat([x,test], sort=False)
x['Mileage'] = x['Mileage'].fillna('17.0 kmpl')
test['Mileage'] = test['Mileage'].fillna('17.0 kmpl')
x['Mileage'] = x['Mileage'].replace("0.0 kmpl", "17.0 kmpl")
test['Mileage'] = test['Mileage'].replace("0.0 kmpl", "17.0 kmpl")
x['Engine'] = x['Engine'].fillna('1197 CC')
test['Engine'] = test['Engine'].fillna('1197 CC')
x['Power'] = x['Power'].fillna('74 bhp')
test['Power'] = test['Power'].fillna('74 bhp')
x['Power'] = x['Power'].replace("null bhp", "74 bhp")
test['Power'] = test['Power'].replace("null bhp", "74 bhp")

In [ ]:

print(x.columns)
print(x.isna().sum())

In [ ]:
print(x.Name.unique())
print(x.Fuel_Type.unique())
print(x.Transmission.unique())
print(x.Owner_Type.unique())
print(x.Location.unique())


In [ ]:
print(x.columns)

In [ ]:
x.info()

In [ ]:
import re

def get_number(name):
    title_search = re.search('([\d+\.+\d]+\W)', name)
    5
    if title_search:
        return title_search.group(1)
    return ""

x['Mileage'] = x['Mileage'].apply(get_number).astype('float')
x['Engine'] = x['Engine'].apply(get_number).astype('int')
x['Power'] = x['Power'].apply(get_number).astype('float')

test['Mileage'] = test['Mileage'].apply(get_number).astype('float')
test['Engine'] = test['Engine'].apply(get_number).astype('int')
test['Power'] = test['Power'].apply(get_number).astype('float')

In [ ]:
x.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
x['Name'] = label_encoder.fit_transform(x['Name'])
x['Location'] = label_encoder.fit_transform(x['Location'])
x['Fuel_Type'] = label_encoder.fit_transform(x['Fuel_Type'])
x['Transmission'] = label_encoder.fit_transform(x['Transmission'])
x['Owner_Type'] = label_encoder.fit_transform(x['Owner_Type'])


In [ ]:
x.info()

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(x,y,train_size=0.8,test_size=0.2,random_state=0)

In [ ]:
X_train.info()

In [ ]:
X_train.head()

In [ ]:
import keras
from keras import layers
import tensorflow as tf
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(X_train.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
EPOCHS = 1000
history = model.fit(
  X_train, y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=1)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
#!pip install git+https://github.com/tensorflow/docs
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)

In [ ]:
import matplotlib.pyplot as plt
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({'Basic': history}, metric = "mae")
plt.ylim([0, 10])
plt.ylabel('MAE [Prices]')

In [ ]:

test['Name'] = label_encoder.fit_transform(test['Name'])
test['Location'] = label_encoder.fit_transform(test['Location'])
test['Fuel_Type'] = label_encoder.fit_transform(test['Fuel_Type'])
test['Transmission'] = label_encoder.fit_transform(test['Transmission'])
test['Owner_Type'] = label_encoder.fit_transform(test['Owner_Type'])
test_predictions = model.predict(test).flatten()

In [ ]:
testsave=pd.read_excel('data_test.xlsx')
testsave['Prices']=test_predictions

In [ ]:
writer = pd.ExcelWriter('testpredictions.xlsx', engine='xlsxwriter')
testsave.to_excel(writer)
writer.save()